In [ ]:
# 查看当前挂载的数据集目录
!ls /home/kesci/input/

In [ ]:
# 查看个人持久化工作区文件
!ls /home/kesci/work/

In [ ]:
# 查看当前kernerl下的package
!pip list --format=columns

In [ ]:
# 显示cell运行时长
%load_ext klab-autotime

In [1]:
# -*- coding:utf-8 -*-

import os
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  
# os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

import tensorflow as tf

from tensorflow.contrib import rnn

import tensorflow.contrib.layers as layers
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit

import tensorflow as tf

import pandas as pd
import numpy as np
from gensim.models import Word2Vec

"""wd_5_bigru_cnn

在论文 Recurrent Convolutional Neural Networks for Text Classification 中。

使用 BiRNN 处理，将每个时刻的隐藏状态和原输入拼起来，在进行 max_pooling 操作。

这里有些不同，首先也是使用 bigru 得到每个时刻的，将每个时刻的隐藏状态和原输入拼起来；

然后使用输入到 TextCNN 网络中。

"""





class Settings(object):

    def __init__(self):


        self.model_name = 'wd_1_2_cnn_max'
        self.query_len = 20
        self.title_len = 20
        self.filter_sizes = 3
        self.n_filter = 200
        self.fc_hidden_size = 512
        self.n_class = 1
        self.train_batch_size=1024
        self.vaild_batch_size=5000
        self.lr=0.01
        self.decay_step=7000
        self.decay_rate=0.8
        self.rnn_hidden_units_hrcn=120






class TEXTCNN(object):

    def __init__(self,  settings):


        self.model_name = settings.model_name
        self.query_len = settings.query_len
        self.title_len = settings.title_len
        self.filter_sizes = settings.filter_sizes
        self.n_filter = settings.n_filter
        self.n_filter_total = self.n_filter 
        self.n_class = settings.n_class
        self.rnn_hidden_units_hrcn=settings.rnn_hidden_units_hrcn
        self.fc_hidden_size = settings.fc_hidden_size
        self._global_step = tf.Variable(0, trainable=False, name='Global_Step')
        self.update_emas = list()
        # placeholders
        self.embedding_size=100
        self.initializer=tf.contrib.layers.xavier_initializer()
        self.hidden_size=120
        self.n_layer_bigru=1
        

        with tf.name_scope('Inputs'):
            self.embedded_x_query=tf.placeholder(tf.float32,[None,self.query_len,self.embedding_size])#h r
            self.embedded_x_title=tf.placeholder(tf.float32,[None,self.title_len,self.embedding_size])# r
            self._y_inputs = tf.placeholder(tf.float32, [None, self.n_class], name='y_input')
            self._keep_prob = tf.placeholder(tf.float32, [])
            self._batch_size = tf.placeholder(tf.int32, [])
            
            self.keep_prob_embed = tf.placeholder(name='keep_prob', dtype=tf.float32)
            self.keep_prob_fully = tf.placeholder(name='keep_prob', dtype=tf.float32)
            self.keep_prob_ae = tf.placeholder(name='keep_prob', dtype=tf.float32)
            self.training = tf.placeholder(name='training', dtype=tf.bool)

        p = self.dropout(self.embedded_x_query, self.keep_prob_embed)
        h = self.dropout(self.embedded_x_title, self.keep_prob_embed)
        

        with tf.variable_scope('bigru_text'):
            output_title = self.bigru_inference(self.embedded_x_query)

        with tf.variable_scope('bigru_content'):
            output_content = self.bigru_inference(self.embedded_x_title)
        
        with tf.name_scope("rnn_output"):
            for i in range(2):
                # BiLSTM
                p_state, h_state = p, h
                for j in range(2):
                    with tf.variable_scope(f'p_lstm_{i}_{j}', reuse=None):
                        p_state, _ = self.BiLSTM(tf.concat(p_state, axis=-1))
                    with tf.variable_scope(f'p_lstm_{i}_{j}' + str(i), reuse=None):
                        h_state, _ = self.BiLSTM(tf.concat(h_state, axis=-1))
    
                p_state = tf.concat(p_state, axis=-1)
                h_state = tf.concat(h_state, axis=-1)
                # attention
                cosine = tf.divide(tf.matmul(p_state, tf.matrix_transpose(h_state)),
                                   (tf.norm(p_state, axis=-1, keep_dims=True) * tf.norm(h_state, axis=-1, keep_dims=True)))
                att_matrix = tf.nn.softmax(cosine)
                p_attention = tf.matmul(att_matrix, h_state)
                h_attention = tf.matmul(att_matrix, p_state)
    
                # DesNet
                p = tf.concat((p, p_state, p_attention), axis=-1)
                h = tf.concat((h, h_state, h_attention), axis=-1)
    
                # auto_encoder
                p = tf.layers.dense(p, 200)
                h = tf.layers.dense(h, 200)
    
                p = self.dropout(p, self.keep_prob_ae)
                h = self.dropout(h, self.keep_prob_ae)
    
            # interaction and prediction layer
            add = p + h
            sub = p - h
            norm = tf.norm(sub, axis=-1)
            out = tf.concat((p, h, add, sub, tf.expand_dims(norm, axis=-1)), axis=-1)
            out = tf.reshape(out, shape=(-1, out.shape[1] * out.shape[2]))
            out = self.dropout(out, self.keep_prob_fully)
    
            out = tf.layers.dense(out, 500, activation='relu')
            out = tf.layers.batch_normalization(out, training=self.training)
            out = tf.layers.dense(out, 500, activation='relu')
            out = tf.layers.batch_normalization(out, training=self.training)
            out = tf.layers.dense(out, 500)
            out = tf.layers.batch_normalization(out, training=self.training)
            self.distance = tf.layers.dense(out, 1)

        with tf.variable_scope('fc-bn-layer'):
            output = tf.concat([output_title, output_content], axis=1)
            W_fc = self.weight_variable([self.hidden_size * 4, self.fc_hidden_size], name='Weight_fc')
            h_fc = tf.matmul(output, W_fc, name='h_fc')
            beta_fc = tf.Variable(tf.constant(0.1, tf.float32, shape=[self.fc_hidden_size], name="beta_fc"))
            fc_bn=tf.layers.batch_normalization(h_fc, training=self.training)
            self.fc_bn_relu = tf.nn.relu(fc_bn, name="relu")

        with tf.variable_scope('out_layer'):
            W_out = self.weight_variable([self.fc_hidden_size, self.n_class], name='Weight_out')
            b_out = self.bias_variable([self.n_class], name='bias_out')
            self._y_pred = tf.nn.xw_plus_b(self.fc_bn_relu, W_out, b_out, name='y_pred')  # 每个类别的分数 scores
            self._y_pred = self._y_pred+self.distance


        with tf.name_scope('loss'):
            self._loss = tf.reduce_mean(
                tf.nn.sigmoid_cross_entropy_with_logits(logits=self._y_pred, labels=self._y_inputs))
            self._y_pred = tf.sigmoid(self._y_pred)


    @property
    def keep_prob(self):
        return self._keep_prob

    @property
    def batch_size(self):
        return self._batch_size

    @property
    def global_step(self):
        return self._global_step


    @property
    def y_inputs(self):
        return self._y_inputs

    @property
    def y_pred(self):
        return self._y_pred

    @property
    def loss(self):
        return self._loss


    def weight_variable(self, shape, name):
        """Create a weight variable with appropriate initialization."""
        initial = tf.truncated_normal(shape, stddev=0.1)
        return tf.Variable(initial, name=name)

    def bias_variable(self, shape, name):
        """Create a bias variable with appropriate initialization."""
        initial = tf.constant(0.1, shape=shape)
        return tf.Variable(initial, name=name)


    def BiLSTM(self, x):
        fw_cell = tf.nn.rnn_cell.BasicLSTMCell(self.rnn_hidden_units_hrcn)
        bw_cell = tf.nn.rnn_cell.BasicLSTMCell(self.rnn_hidden_units_hrcn)
        return tf.nn.bidirectional_dynamic_rnn(fw_cell, bw_cell, x, dtype=tf.float32)
        
    def dropout(self, x, keep_prob):
        return tf.nn.dropout(x, keep_prob)
        
    def gru_cell(self):
        with tf.name_scope('gru_cell'):
            cell = rnn.GRUCell(self.hidden_size, reuse=tf.get_variable_scope().reuse)
        return rnn.DropoutWrapper(cell, output_keep_prob=self.keep_prob)

    def bi_gru(self, inputs):
        """build the bi-GRU network. 返回个所有层的隐含状态。"""
        cells_fw = [self.gru_cell() for _ in range(self.n_layer_bigru)]
        cells_bw = [self.gru_cell() for _ in range(self.n_layer_bigru)]
        initial_states_fw = [cell_fw.zero_state(self.batch_size, tf.float32) for cell_fw in cells_fw]
        initial_states_bw = [cell_bw.zero_state(self.batch_size, tf.float32) for cell_bw in cells_bw]
        outputs, _, _ = rnn.stack_bidirectional_dynamic_rnn(cells_fw, cells_bw, inputs,
                                                            initial_states_fw=initial_states_fw,
                                                            initial_states_bw=initial_states_bw, dtype=tf.float32)
        return outputs

    def task_specific_attention(self, inputs, output_size,
                                initializer=layers.xavier_initializer(),
                                activation_fn=tf.tanh, scope=None):
        assert len(inputs.get_shape()) == 3 and inputs.get_shape()[-1].value is not None
        with tf.variable_scope(scope or 'attention') as scope:
            # u_w, attention 向量
            attention_context_vector = tf.get_variable(name='attention_context_vector', shape=[output_size],
                                                       initializer=initializer, dtype=tf.float32)
            # 全连接层，把 h_i 转为 u_i ， shape= [batch_size, units, input_size] -> [batch_size, units, output_size]
            input_projection = layers.fully_connected(inputs, output_size, activation_fn=activation_fn, scope=scope)
            # 输出 [batch_size, units]
            vector_attn = tf.reduce_sum(tf.multiply(input_projection, attention_context_vector), axis=2, keep_dims=True)
            attention_weights = tf.nn.softmax(vector_attn, dim=1)
            tf.summary.histogram('attention_weigths', attention_weights)
            weighted_projection = tf.multiply(inputs, attention_weights)
            outputs = tf.reduce_sum(weighted_projection, axis=1)
            return outputs  # 输出 [batch_size, hidden_size*2]


    def bigru_inference(self, X_inputs):
        inputs=X_inputs
        output_bigru = self.bi_gru(inputs)
        output_att = self.task_specific_attention(output_bigru, self.hidden_size*2)
        return output_att



def loader(data_store_path='first_zzp/'):############
    # all_data=pd.read_csv(data_store_path+"data_sets/train_feature_second_no_query_and_title_data.csv")
    # query_title=pd.read_csv(data_store_path+"data_sets/train_data.csv",usecols=['query','title'])
    print("开始加载数据")
    all_data=pd.read_pickle(data_store_path+"train_data.pickle")
    data_split=StratifiedShuffleSplit(n_splits=2,test_size=0.001,random_state=1)
    train_index,vaild_index=data_split.split(all_data['label'],all_data['label']).__next__()
    train_data=all_data.iloc[train_index]
    vaild_data=all_data.iloc[vaild_index]
    print("加载数据完成")
    return train_data,vaild_data

def batch_iter(data_x,data_y, batch_size, num_epochs, shuffle=True):#这个就是产生batch的，可以直接用
    """
    Generates a batch iterator for a dataset.
    """
    # data_x=data_x.tolist()
    data=list(zip(data_x,data_y))
    data=np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:#如果为true 则代表允许随机取数据
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]#随机取数据
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]#而不用一个个加到列表里了。这是一个batch，嗯哼
            #取到的维度为（batchsize,2） y为(batchsize,1) 里面存的是序号

def get_w2v_array(word_list,max_len):
    array = np.zeros((max_len, 100))
    if len(word_list)<=max_len:
        for i in range(len(word_list)):
            if str(word_list[i]) in word_wv.vocab.keys():
                array[i][:] = word_wv[str(word_list[i])]
    else:
        for i in range(max_len):
            if str(word_list[i]) in word_wv.vocab.keys():
                array[i][:] = word_wv[str(word_list[i])]
    return array


settings = Settings()

train_batches,vaild_data=loader()

train_batches=np.array(train_batches)
train_batches_y=train_batches[:,2]
train_batches_x=np.delete(train_batches, 2, axis=1)######这里是除了label以外的维数


batches=batch_iter(train_batches_x,train_batches_y,settings.train_batch_size,4)

vaild_batches=np.array(vaild_data)
vaild_batches_y=vaild_batches[:,2]
vaild_batches_x=np.delete(vaild_batches, 2, axis=1)
vaild_data=batch_iter(vaild_batches_x,vaild_batches_y,settings.vaild_batch_size,1000)

print("train starting")
w2v_model = Word2Vec.load('w2v_model/w2v_all_data_model.txt')
word_wv= w2v_model.wv


settings = Settings()

settings.all_features_num=train_batches_x.shape[1]-2###########自动设置数值型特征数量
model = TEXTCNN( settings)

learning_rate = tf.train.exponential_decay(settings.lr, model.global_step, settings.decay_step,
                                       settings.decay_rate, staircase=True)
optimizer = tf.train.AdamOptimizer(learning_rate)
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    grads_and_vars=optimizer.compute_gradients(model.loss)
    train_op=optimizer.apply_gradients(grads_and_vars,global_step=model.global_step)
sess = tf.Session() #启动创建的模型
sess.run(tf.initialize_all_variables())


# saver.restore(sess, tf.train.latest_checkpoint('tf_model_rcnn/'))
per_train_loss_avg_list=[]
per_vaild_loss_avg_list=[]
parts_time=[]
from time import time
for batch in batches:
    time1=time()
    batch_x,batch_y=zip(*batch)
    batch_y=np.array(batch_y)[:,None]
    batch_x=[i.tolist() for i in batch_x]
    batch_x=pd.DataFrame(batch_x)
    batch_x[[0,1]]=batch_x[[0,1]].applymap(lambda x:x.split(' '))
    batch_x=np.array(batch_x)
    #####[None,title_len,embedding_size]
    #np.apply_along_axis(lambda x:[5,5,5,5,5] if x[0]==2 else x,axis=2,arr=a)
    #query   title   title_count         title_ctr      label
    batch_x_query=np.apply_along_axis(lambda x:get_w2v_array(x[0],settings.query_len),axis=1,arr=batch_x)
    batch_x_title=np.apply_along_axis(lambda x:get_w2v_array(x[1],settings.title_len),axis=1,arr=batch_x)
    #     train_op=optimizer.apply_gradients(grads_and_vars,global_step=global_step)                              
    feed_dict = {model.embedded_x_query: batch_x_query,
                 model.embedded_x_title: batch_x_title,
                 model.y_inputs: batch_y,
                 model.batch_size: batch_x.shape[0], 
                 model.keep_prob: 0.8,
        
                model.keep_prob_embed:0.8,
                model.keep_prob_fully:0.8,
                model.keep_prob_ae:0.5,
                model.training:True,
        
    }

    _,step,loss_out=sess.run([train_op,model.global_step,model.loss],feed_dict)
    time2=time()
    parts_time.append(time2-time1)
    if step%10==0:
        print("step {},loss:{:g}".format(step,loss_out))              
    per_train_loss_avg_list.append(loss_out)
    
    
    if step%(int(50000000/1004))==0:
        print("batch ended")
    if step%500==0:
        time_single_part=sum(parts_time)
        ed_time=(50000000/settings.train_batch_size-step)/500*time_single_part
        print("距离训练完一整轮剩余时间：",ed_time/60," 分钟")
        parts_time=[]
        vaild_x,vaild_y=zip(*(vaild_data.__next__()))
        vaild_y=np.array(vaild_y)[:,None]
        vaild_x=[i.tolist() for i in vaild_x]
        vaild_x=pd.DataFrame(vaild_x)
        vaild_x[[0,1]]=vaild_x[[0,1]].applymap(lambda x:x.split(' '))
        vaild_x=np.array(vaild_x)
    
        vaild_x_query=np.apply_along_axis(lambda x:get_w2v_array(x[0],settings.query_len),axis=1,arr=vaild_x)
        vaild_x_title=np.apply_along_axis(lambda x:get_w2v_array(x[1],settings.title_len),axis=1,arr=vaild_x)

        feed_dict = {model.embedded_x_query: vaild_x_query,
                     model.embedded_x_title: vaild_x_title,
                     model.y_inputs: vaild_y,
                     model.batch_size: vaild_x.shape[0], 
                     model.keep_prob: 1,

                    model.keep_prob_embed:1,
                    model.keep_prob_fully:1,
                    model.keep_prob_ae:1,
                    model.training:False,
            
            
        }
    
        step,loss_out=sess.run([model.global_step,model.loss],feed_dict)
        per_vaild_loss_avg_list.append(loss_out)
        per_train_loss_avg=sum(per_train_loss_avg_list)/len(per_train_loss_avg_list)
        print("per_train_loss_avg:",per_train_loss_avg,"**********")
        per_train_loss_avg_list=[]
        print("vaild_loss:",loss_out)
    if step%5000==0:
        per_vaild_loss_avg=sum(per_vaild_loss_avg_list)/len(per_vaild_loss_avg_list)
        per_vaild_loss_avg_list=[]
        print("per_vaild_loss_avg:",per_vaild_loss_avg,"************")
        saver = tf.train.Saver(max_to_keep = 3,var_list=tf.global_variables())
        saver = saver.save(sess, "/home/kesci/work/first_zzp/tf_model_bigru_hrcn/rnn", global_step=step)

开始加载数据
加载数据完成
train starting


/opt/conda/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
dim is deprecated, use axis instead
Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Use `tf.global_variables_initializer` instead.
step 10,loss:2.31439
step 20,loss:0.795758
step 30,loss:0.559962
step 40,loss:0.491966
step 50,loss:0.483351
step 60,loss:0.489116
step 70,loss:0.468662
step 80,loss:0.472852
step 90,loss:0.494204
step 100,loss:0.531546
step 110,loss:0.448226
step 120,loss:0.498237
step 130,loss:0.49694
step 140,loss:0.501911
step 150,loss:0.515829
step 160,loss:0.497761
step 170,loss:0.459991
step 180,loss:0.508972
step 190,loss:0.468498
step 200,loss:0.497193
step 210,loss:0.492086
step 220,loss

In [2]:
!fuser -v /dev/nvidia*

                     USER        PID ACCESS COMMAND
/dev/nvidia-uvm:     root     kernel mount /dev/nvidia-uvm
                     kesci      2501 F.... python
/dev/nvidia-uvm-tools:
                     root     kernel mount /dev/nvidia-uvm-tools
/dev/nvidia0:        root     kernel mount /dev/nvidia0
                     kesci      2501 F...m python
/dev/nvidiactl:      root     kernel mount /dev/nvidiactl
                     kesci      2501 F...m python


In [3]:
!kill -9 2501

In [2]:
parts_time=[]
from time import time
for batch in batches:
    time1=time()
    batch_x,batch_y=zip(*batch)
    batch_y=np.array(batch_y)[:,None]
    batch_x=[i.tolist() for i in batch_x]
    batch_x=pd.DataFrame(batch_x)
    batch_x[[0,1]]=batch_x[[0,1]].applymap(lambda x:x.split(' '))
    batch_x=np.array(batch_x)
    #####[None,title_len,embedding_size]
    #np.apply_along_axis(lambda x:[5,5,5,5,5] if x[0]==2 else x,axis=2,arr=a)
    #query   title   title_count         title_ctr      label
    batch_x_query=np.apply_along_axis(lambda x:get_w2v_array(x[0],settings.query_len),axis=1,arr=batch_x)
    batch_x_title=np.apply_along_axis(lambda x:get_w2v_array(x[1],settings.title_len),axis=1,arr=batch_x)
    #     train_op=optimizer.apply_gradients(grads_and_vars,global_step=global_step)                              
    feed_dict = {model.embedded_x_query: batch_x_query,
                 model.embedded_x_title: batch_x_title,
                 model.y_inputs: batch_y,
                 model.batch_size: batch_x.shape[0], 
                 model.keep_prob: 0.8,
        
                model.keep_prob_embed:0.8,
                model.keep_prob_fully:0.8,
                model.keep_prob_ae:0.5,
                model.training:True,
        
    }

    _,step,loss_out=sess.run([train_op,model.global_step,model.loss],feed_dict)
    time2=time()
    parts_time.append(time2-time1)
    if step%10==0:
        print("step {},loss:{:g}".format(step,loss_out))              
    per_train_loss_avg_list.append(loss_out)
    
    
    if step%(int(50000000/1004))==0:
        print("batch ended")
    if step%500==0:
        time_single_part=sum(parts_time)
        ed_time=(50000000/settings.train_batch_size-step)/500*time_single_part
        print("距离训练完一整轮剩余时间：",ed_time/60," 分钟")
        parts_time=[]
        vaild_x,vaild_y=zip(*(vaild_data.__next__()))
        vaild_y=np.array(vaild_y)[:,None]
        vaild_x=[i.tolist() for i in vaild_x]
        vaild_x=pd.DataFrame(vaild_x)
        vaild_x[[0,1]]=vaild_x[[0,1]].applymap(lambda x:x.split(' '))
        vaild_x=np.array(vaild_x)
    
        vaild_x_query=np.apply_along_axis(lambda x:get_w2v_array(x[0],settings.query_len),axis=1,arr=vaild_x)
        vaild_x_title=np.apply_along_axis(lambda x:get_w2v_array(x[1],settings.title_len),axis=1,arr=vaild_x)

        feed_dict = {model.embedded_x_query: vaild_x_query,
                     model.embedded_x_title: vaild_x_title,
                     model.y_inputs: vaild_y,
                     model.batch_size: vaild_x.shape[0], 
                     model.keep_prob: 1,

                    model.keep_prob_embed:1,
                    model.keep_prob_fully:1,
                    model.keep_prob_ae:1,
                    model.training:False,
            
            
        }
    
        step,loss_out=sess.run([model.global_step,model.loss],feed_dict)
        per_vaild_loss_avg_list.append(loss_out)
        per_train_loss_avg=sum(per_train_loss_avg_list)/len(per_train_loss_avg_list)
        print("per_train_loss_avg:",per_train_loss_avg,"**********")
        per_train_loss_avg_list=[]
        print("vaild_loss:",loss_out)
    if step%5000==0:
        per_vaild_loss_avg=sum(per_vaild_loss_avg_list)/len(per_vaild_loss_avg_list)
        per_vaild_loss_avg_list=[]
        print("per_vaild_loss_avg:",per_vaild_loss_avg,"************")
        saver = tf.train.Saver(max_to_keep = 3,var_list=tf.global_variables())
        saver = saver.save(sess, "/home/kesci/work/first_zzp/tf_model_bigru_hrcn/rnn", global_step=step)

step 7300,loss:0.475182
step 7310,loss:0.48214
step 7320,loss:0.442797
step 7330,loss:0.464975
step 7340,loss:0.47957
step 7350,loss:0.463268
step 7360,loss:0.451333
step 7370,loss:0.443255
step 7380,loss:0.472878
step 7390,loss:0.475085
step 7400,loss:0.4738
step 7410,loss:0.512576
step 7420,loss:0.449731
step 7430,loss:0.427682
step 7440,loss:0.490056
step 7450,loss:0.432677
step 7460,loss:0.444967
step 7470,loss:0.457567
step 7480,loss:0.463213
step 7490,loss:0.465898
step 7500,loss:0.45297
距离训练完一整轮剩余时间： 200.7374793558071  分钟
per_train_loss_avg: 0.4633075876799757 **********
vaild_loss: 0.45274818
step 7510,loss:0.484436
step 7520,loss:0.466174
step 7530,loss:0.452982
step 7540,loss:0.439087
step 7550,loss:0.494937
step 7560,loss:0.443573
step 7570,loss:0.467521
step 7580,loss:0.49258
step 7590,loss:0.443348
step 7600,loss:0.452766
step 7610,loss:0.498642
step 7620,loss:0.478847
step 7630,loss:0.495396
step 7640,loss:0.482614
step 7650,loss:0.444225
step 7660,loss:0.477744
step 7670